In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import plotly.graph_objs as go

#### В этом задании вам необходимо:
1. Реализовать t-критерий Стьюдента для 2 независимых выборок при условии неизвестной дисперсии. Необходимо считать значение статистики и p_value для разных видов гипотез (двусторонняя, односторонние), а также результат (отвергается нулевая гипотеза или нет). Сравнить результаты с реализацией в scipy.stats.ttest_ind. 
2. Реализовать bootstrap для оценки (через доверительные интервалы) медианных и средних значений распределений. Рассчитать для распределений: нормальное, экспоненциальное, смесь нормальных
3. Рассчитать мощность критерия для t-критерия и критерия Манна-Уитни для разных распределений и разном эффекте. Отдельно изучить случай, когда средние равны, а дисперсии сильно отличаются.
4. Оценить корректность t-критерия и критерия Манна-Уитни на разных распределениях.
#### Задание принимается в jupyter notebook
------
1. Реализовать t-критерий Стьюдента для 2 независимых выборок при условии неизвестной дисперсии. Необходимо считать значение статистики и p_value для разных видов гипотез (двусторонняя, односторонние), а также результат (отвергается нулевая гипотеза или нет). Сравнить результаты с реализацией в scipy.stats.ttest_ind.

In [2]:
MEAN_LOW = 10
MEAN_HIGH = 15
STD_LOW = 1
STD_HIGH = 5

def create_samples_pair(size=100):
    ma, mb = np.random.randint(MEAN_LOW, MEAN_HIGH, size=(2,))
    sa, sb = np.random.randint(STD_LOW, STD_HIGH, size=(2,))
    xa = sps.norm.rvs(loc=ma, scale=sa, size=size)
    xb = sps.norm.rvs(loc=mb, scale=sb, size=size)
    print(f"A-group params: mean = {ma}, std = {sa}")
    print(f"B-group params: mean = {mb}, std = {sb}")
    return xa, xb

In [3]:
def t_test_df(s1, n1, s2, n2):

    v1 = n1 - 1
    v2 = n2 - 1

    num = (s1**2 / n1 + s2**2 / n2) ** 2
    denom = s1**4 / (n1**2 * v1) + s2**4 / (n2**2 * v2)
    return round(np.divide(num, denom))


def t_test(xa, xb, alternative="two-sided"):
    m_xa = np.mean(xa)
    m_xb = np.mean(xb)
    s_xa = np.std(xa, ddof=1)
    s_xb = np.std(xb, ddof=1)
    na = len(xa)
    nb = len(xb)
    new_s_xa = s_xa / np.sqrt(na)
    new_s_xb = s_xb / np.sqrt(nb)
    t = (m_xa - m_xb) / np.sqrt(new_s_xa**2 + new_s_xb**2)
    df = t_test_df(s_xa, na, s_xb, nb)
    if alternative == "less":
        pval = sps.t.sf(-t, df=df)
    elif alternative == "greater":
        pval = sps.t.sf(t, df=df)
    elif alternative == "two-sided":
        pval = sps.t.sf(abs(t), df=df) * 2
    else:
        raise ValueError("alternative must be " "'less', 'greater' or 'two-sided'")
    return t, pval

In [4]:
def compare_tests(alternative="two-sided"):

    xa, xb = create_samples_pair(size=100)
    for alternative in ("less", "greater", "two-sided"):
        custom_ttest = t_test(xa, xb, alternative=alternative)
        sps_ttest = sps.ttest_ind(xa, xb, equal_var=False, alternative=alternative)
        np.testing.assert_almost_equal(custom_ttest[0], sps_ttest[0], decimal=5)
        np.testing.assert_almost_equal(custom_ttest[1], sps_ttest[1], decimal=5)
        print(f"[alternative = {alternative}] Done!")

In [5]:
for _ in range(5):
    compare_tests()

A-group params: mean = 10, std = 2
B-group params: mean = 10, std = 2
[alternative = less] Done!
[alternative = greater] Done!
[alternative = two-sided] Done!
A-group params: mean = 12, std = 1
B-group params: mean = 10, std = 4
[alternative = less] Done!
[alternative = greater] Done!
[alternative = two-sided] Done!
A-group params: mean = 14, std = 1
B-group params: mean = 10, std = 4
[alternative = less] Done!
[alternative = greater] Done!
[alternative = two-sided] Done!
A-group params: mean = 12, std = 1
B-group params: mean = 12, std = 1
[alternative = less] Done!
[alternative = greater] Done!
[alternative = two-sided] Done!
A-group params: mean = 11, std = 2
B-group params: mean = 12, std = 4
[alternative = less] Done!
[alternative = greater] Done!
[alternative = two-sided] Done!


------
2. Реализовать bootstrap для оценки (через доверительные интервалы) медианных и средних значений распределений. Рассчитать для распределений: нормальное, экспоненциальное, смесь нормальных

In [6]:
NORM_MEAN_LOW = 10
NORM_MEAN_HIGH = 15
NORM_STD_LOW = 1
NORM_STD_HIGH = 5

EXP_LOC_LOW = 10
EXP_LOC_HIGH = 15
EXP_SCALE_LOW = 1
EXP_SCALE_HIGH = 5

In [7]:
def bootstrap_samples(x, samples_num):
    idx = np.random.randint(0, len(x), (samples_num, len(x)))
    samples = x[idx]
    return samples


def bootstrap_percentile(x_bootstraped, metric, alpha=0.05):
    metric_values = metric(x_bootstraped, axis=1)
    percentile = np.percentile(metric_values, (alpha * 100, (1 - alpha) * 100))
    return percentile


def estimate_and_compare(x, samples_num=500, alpha=0.05):
    for metric, name in zip((np.mean, np.median), ("mean", "median")):
        percentile = bootstrap_percentile(
            bootstrap_samples(x, samples_num), metric, alpha=alpha
        )
        real = round(metric(x), 3)
        low = round(percentile[0], 3)
        high = round(percentile[1], 3)
        assert real >= low
        assert real <= high
        print(f"Real {name} = {real}")
        print(f"CI: low = {low}, high = {high}")


def pipe(size=100, type_="norm", samples_num=500, alpha=0.05):
    if type_ == "norm":
        m = np.random.randint(NORM_MEAN_LOW, NORM_MEAN_HIGH)
        s = np.random.randint(NORM_STD_LOW, NORM_STD_HIGH)
        x = sps.norm.rvs(loc=m, scale=s, size=size)
        print(f"{type_} params: mean = {m}, std = {s}")
    elif type_ == "exp":
        l = np.random.randint(EXP_LOC_LOW, EXP_LOC_HIGH)
        s = np.random.randint(EXP_SCALE_LOW, EXP_SCALE_HIGH)
        x = sps.expon.rvs(loc=l, scale=s, size=size)
        print(f"{type_} params: loc = {l}, scale = {s}")
    elif type_ == "2norm":
        x = np.array([])
        for i in range(2):
            m = np.random.randint(NORM_MEAN_LOW, NORM_MEAN_HIGH)
            s = np.random.randint(NORM_STD_LOW, NORM_STD_HIGH)
            temp_x = sps.norm.rvs(loc=m, scale=s, size=size)
            x = np.concatenate([x, temp_x])
            print(f"{type_} params: mean{i+1} = {m}, std{i+1} = {s}")
    estimate_and_compare(x, samples_num=samples_num, alpha=alpha)

In [8]:
for _ in range(3):
    pipe()

norm params: mean = 11, std = 4
Real mean = 11.118
CI: low = 10.527, high = 11.747
Real median = 11.286
CI: low = 10.513, high = 11.968
norm params: mean = 11, std = 2
Real mean = 10.87
CI: low = 10.527, high = 11.194
Real median = 10.668
CI: low = 10.057, high = 11.187
norm params: mean = 10, std = 2
Real mean = 9.803
CI: low = 9.522, high = 10.08
Real median = 9.815
CI: low = 9.599, high = 10.149


In [9]:
for _ in range(3):
    pipe(type_='exp')

exp params: loc = 13, scale = 2
Real mean = 15.127
CI: low = 14.792, high = 15.471
Real median = 14.368
CI: low = 13.971, high = 14.693
exp params: loc = 11, scale = 2
Real mean = 12.811
CI: low = 12.462, high = 13.169
Real median = 12.362
CI: low = 11.862, high = 12.563
exp params: loc = 11, scale = 2
Real mean = 12.77
CI: low = 12.443, high = 13.134
Real median = 12.106
CI: low = 11.864, high = 12.36


In [10]:
for _ in range(3):
    pipe(type_='2norm')

2norm params: mean1 = 13, std1 = 3
2norm params: mean2 = 10, std2 = 3
Real mean = 11.357
CI: low = 10.964, high = 11.738
Real median = 11.074
CI: low = 10.719, high = 11.469
2norm params: mean1 = 14, std1 = 2
2norm params: mean2 = 14, std2 = 3
Real mean = 14.183
CI: low = 13.918, high = 14.468
Real median = 14.001
CI: low = 13.779, high = 14.425
2norm params: mean1 = 13, std1 = 3
2norm params: mean2 = 12, std2 = 1
Real mean = 12.63
CI: low = 12.378, high = 12.905
Real median = 12.259
CI: low = 11.944, high = 12.488



------
3. Рассчитать мощность критерия для t-критерия и критерия Манна-Уитни для разных распределений и разном эффекте. Отдельно изучить случай, когда средние равны, а дисперсии сильно отличаются.

In [11]:
LOC_LOW = 10
LOC_HIGH = 15
SCALE_LOW = 10
SCALE_HIGH = 20

EFFECT_LOC_LOW = 5
EFFECT_LOC_HIGH = 10
EFFECT_SCALE_LOW = 10
EFFECT_SCALE_HIGH = 50

In [12]:
def power(p_values, alpha=0.05):
    num = len(p_values[p_values < alpha])
    denom = len(p_values)
    return np.divide(num, denom) 

In [13]:
def iter_(distr, test, loc, scale, loc_effect, scale_effect, size=100, norm=True):
    xa = distr.rvs(loc=loc, scale=scale, size=size)
    xb = distr.rvs(loc=loc + loc_effect, scale=scale + scale_effect, size=size)
    if norm:
        p_value = test(xa, xb, equal_var=False).pvalue
    else:
        p_value = test(xa, xb).pvalue
    return p_value


def experiment(
    distr, size=100, iters=100, alpha=0.05, state_loc=False, state_scale=False
):
    loc = np.random.randint(LOC_LOW, LOC_HIGH)
    scale = np.random.randint(SCALE_LOW, SCALE_HIGH)
    loc_effect = 0 if state_loc else np.random.randint(EFFECT_LOC_LOW, EFFECT_LOC_HIGH)
    scale_effect = (
        0 if state_scale else np.random.randint(EFFECT_SCALE_LOW, EFFECT_SCALE_HIGH)
    )
    print(f"Параметры распределения: loc = {loc}, scale = {scale}")
    print(
        f"Параметры эффекта: loc_effect = {loc_effect}, scale_effect = {scale_effect}"
    )
    p_values = np.array(
        [
            iter_(
                distr,
                sps.ttest_ind,
                loc,
                scale,
                loc_effect,
                scale_effect,
                size,
                norm=True,
            )
            for _ in range(iters)
        ]
    )
    pw = power(p_values, alpha=alpha)
    print(f"Мощность t-test: {pw}")
    p_values = np.array(
        [
            iter_(
                distr,
                sps.mannwhitneyu,
                loc,
                scale,
                loc_effect,
                scale_effect,
                size,
                norm=False,
            )
            for _ in range(iters)
        ]
    )
    pw = power(p_values, alpha=alpha)
    print(f"Мощность Mann-Whitney: {pw}")

In [14]:
# norm with different effect
for _ in range(3):
    res = experiment(sps.norm)
    print("-----------------------------------")

Параметры распределения: loc = 10, scale = 16
Параметры эффекта: loc_effect = 7, scale_effect = 22
Мощность t-test: 0.39
Мощность Mann-Whitney: 0.41
-----------------------------------
Параметры распределения: loc = 13, scale = 14
Параметры эффекта: loc_effect = 9, scale_effect = 34
Мощность t-test: 0.45


Мощность Mann-Whitney: 0.41
-----------------------------------
Параметры распределения: loc = 11, scale = 16
Параметры эффекта: loc_effect = 7, scale_effect = 18
Мощность t-test: 0.42
Мощность Mann-Whitney: 0.42
-----------------------------------


In [15]:
# norm with state loc
for _ in range(3):
    res = experiment(sps.norm, state_loc=True)
    print("-----------------------------------")

Параметры распределения: loc = 11, scale = 16
Параметры эффекта: loc_effect = 0, scale_effect = 14
Мощность t-test: 0.04
Мощность Mann-Whitney: 0.06
-----------------------------------
Параметры распределения: loc = 12, scale = 15
Параметры эффекта: loc_effect = 0, scale_effect = 22
Мощность t-test: 0.06
Мощность Mann-Whitney: 0.05
-----------------------------------
Параметры распределения: loc = 12, scale = 17
Параметры эффекта: loc_effect = 0, scale_effect = 43
Мощность t-test: 0.06
Мощность Mann-Whitney: 0.06
-----------------------------------


In [16]:
# norm with state scale
for _ in range(3):
    res = experiment(sps.norm, state_scale=True)
    print("-----------------------------------")

Параметры распределения: loc = 12, scale = 17
Параметры эффекта: loc_effect = 6, scale_effect = 0
Мощность t-test: 0.61
Мощность Mann-Whitney: 0.68
-----------------------------------
Параметры распределения: loc = 11, scale = 18
Параметры эффекта: loc_effect = 5, scale_effect = 0
Мощность t-test: 0.55
Мощность Mann-Whitney: 0.45
-----------------------------------
Параметры распределения: loc = 12, scale = 14
Параметры эффекта: loc_effect = 8, scale_effect = 0
Мощность t-test: 0.98
Мощность Mann-Whitney: 0.98
-----------------------------------


In [17]:
# exp with different effects
for _ in range(3):
    res = experiment(sps.expon)
    print("-----------------------------------")

Параметры распределения: loc = 10, scale = 19
Параметры эффекта: loc_effect = 8, scale_effect = 21
Мощность t-test: 1.0


Мощность Mann-Whitney: 1.0
-----------------------------------
Параметры распределения: loc = 10, scale = 18
Параметры эффекта: loc_effect = 9, scale_effect = 13
Мощность t-test: 1.0
Мощность Mann-Whitney: 1.0
-----------------------------------
Параметры распределения: loc = 13, scale = 17
Параметры эффекта: loc_effect = 5, scale_effect = 25
Мощность t-test: 1.0
Мощность Mann-Whitney: 1.0
-----------------------------------


In [18]:
# exp with state loc
for _ in range(3):
    res = experiment(sps.expon, state_loc=True)
    print("-----------------------------------")

Параметры распределения: loc = 10, scale = 16
Параметры эффекта: loc_effect = 0, scale_effect = 44
Мощность t-test: 1.0
Мощность Mann-Whitney: 1.0
-----------------------------------
Параметры распределения: loc = 14, scale = 17
Параметры эффекта: loc_effect = 0, scale_effect = 18
Мощность t-test: 1.0
Мощность Mann-Whitney: 1.0
-----------------------------------
Параметры распределения: loc = 13, scale = 18
Параметры эффекта: loc_effect = 0, scale_effect = 14
Мощность t-test: 0.97
Мощность Mann-Whitney: 0.96
-----------------------------------


In [19]:
# exp with state scale
for _ in range(3):
    res = experiment(sps.norm, state_scale=True)
    print("-----------------------------------")

Параметры распределения: loc = 14, scale = 10
Параметры эффекта: loc_effect = 7, scale_effect = 0
Мощность t-test: 1.0
Мощность Mann-Whitney: 0.99
-----------------------------------
Параметры распределения: loc = 13, scale = 19
Параметры эффекта: loc_effect = 5, scale_effect = 0
Мощность t-test: 0.47
Мощность Mann-Whitney: 0.57
-----------------------------------
Параметры распределения: loc = 11, scale = 13
Параметры эффекта: loc_effect = 8, scale_effect = 0
Мощность t-test: 0.99
Мощность Mann-Whitney: 0.99
-----------------------------------


------
4. Оценить корректность t-критерия и критерия Манна-Уитни на разных распределениях.

In [20]:
LOC_LOW = 10
LOC_HIGH = 15
SCALE_LOW = 10
SCALE_HIGH = 20

In [21]:
def plot(p_values, test_name, iters=1000):
    probs = []
    step = 1 / iters
    x = [step * i for i in range(iters + 1)]
    for i in range(iters + 1):
        alpha_step = step * i
        probs.append(p_values[p_values < alpha_step].shape[0] / p_values.shape[0])
    fig = go.Figure(
        [
            go.Scatter(x=x, y=probs, mode="markers", name="p_value"),
            go.Scatter(x=x, y=x, mode="lines", name="uniform"),
        ]
    )
    fig.update_layout(height=400, width=400, title=f"Q-Q plot for {test_name}")
    fig.show()

In [22]:
def p_values(distr, size=100, iters=1000):
    p_values_ttest = []
    p_values_mannwhitneyu = []
    for i in range(iters):
        loc = np.random.randint(LOC_LOW, LOC_HIGH)
        scale = np.random.randint(SCALE_LOW, SCALE_HIGH)
        xa = distr.rvs(loc=loc, scale=scale, size=size)
        xb = distr.rvs(loc=loc, scale=scale, size=size)

        p_value_ttest = sps.ttest_ind(xa, xb, equal_var=False).pvalue
        p_values_ttest.append(p_value_ttest)
        p_value_mannwhitneyu = sps.mannwhitneyu(xa, xb).pvalue
        p_values_mannwhitneyu.append(p_value_mannwhitneyu)
    p_values_ttest = np.array(p_values_ttest)
    p_values_mannwhitneyu = np.array(p_values_mannwhitneyu)
    return p_values_ttest, p_values_mannwhitneyu

In [23]:
# norm
iters = 1000
size = 100
p_values_ttest, p_values_mannwhitneyu = p_values(sps.norm, iters=iters, size=size)

In [24]:
# norm
plot(p_values_ttest, 'T-test', iters=iters)

In [25]:
# norm
plot(p_values_mannwhitneyu, 'Mann-Whitney', iters=iters)

In [26]:
# expon
iters = 1000
size = 100
p_values_ttest, p_values_mannwhitneyu = p_values(sps.expon, iters=iters, size=size)

In [27]:
# expon
plot(p_values_ttest, 'T-test', iters=iters)

In [28]:
# expon
plot(p_values_mannwhitneyu, 'Mann-Whitney', iters=iters)